###### The objective of this program is the following:
1. Extract monthly market data for 10 tickers from Alpha Vantage API
2. Isolate Close prices and develop a function that accepts prices, sector, date, top number of prices and returns top industries for that date.

###### Activating code auto-complete

In [164]:
%config IPCompleter.greedy=True

###### Importing Libraries

In [163]:
from alpha_vantage.timeseries import TimeSeries
import time
import pandas as pd

###### Parsing API Key from credentials file

In [ ]:
with open("api_key.cred", "r") as cred_file:
    for line in cred_file:
        api_key = line.split("=")[1]

###### Storing List of Tickers and Connecting to the API

In [ ]:
# List of companies
list_companies = ['AAPL','MSFT','AMZN','FB','BRK.B','JNJ','GOOG','JPM','V','PG']

# Chose your output format, or default to JSON (python dict)
ts = TimeSeries(api_key, output_format='pandas')
ti = TechIndicators(api_key)


###### Defining Sector for Tickers

In [ ]:
# Sector for each company
dict_sector = {'AAPL':'Technology','MSFT':'Technology', 'AMZN':'Retail',
               'FB':'Technology','BRK.B':'Financial Services','JNJ':'Pharmaceuticals',
               'GOOG':'Technology','JPM':'Financial Services','V':'Financial Services',
                'PG':'Industrial'}

sector_data = []
for k,v in dict_sector.items():
    sector_data.append(v)
    
s_sector = pd.Series(data = sector_data, index = list_companies)
s_sector
    

###### Extracting Data for each Ticker from API

In [ ]:
df_list = []
for company in list_companies:
    data, metadata = ts.get_monthly(symbol=company)
    data = pd.DataFrame(data['4. close'])
    data.rename(columns = {'4. close':company}, inplace = True)
    df_list.append(data)
#     print(df_list)
    time.sleep(21)
    

###### Concatenating Ticker Outputs to form final DataFrame

In [ ]:
df_prices = pd.concat(df_list,axis=1,sort=True)
df_prices
# for i in df_prices.index:
#     print(i)

###### Testing lookups against sector series [DEV]

In [167]:
# close prices for all tickers on given date
df_prices.loc['2004-02-27']

# tickers with highest 2 close prices
list(df_prices.loc['2004-02-27'].nlargest(2).index)

# looking up against sector series
list(s_sector.loc[list(df_prices.loc['2004-02-27'].nlargest(2).index)])

['Financial Services', 'Industrial']

###### Defining Final Function

In [ ]:
def date_top_industries(prices, sector, date, top_n):
    """
    Get the set of the top industries for the date
    
    Parameters
    ----------
    prices : DataFrame
        Prices for each ticker and date
    sector : Series
        Sector name for each ticker
    date : Date
        Date to get the top performers
    top_n : int
        Number of top performers to get
    
    Returns
    -------
    top_industries : set
        Top industries for the date
    """
    # TODO: Implement Function
    close_prices = prices.loc[date]
    top_n_tickers = list(close_prices.nlargest(top_n).index)
    sector_lookup = sector.loc[top_n_tickers]
    
    return set(sector_lookup)

###### Calling Final Function

In [ ]:
date_top_industries(df_prices, s_sector, '2008-09-30', 2)